<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/ML801-IDONTEVENKNOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers
!pip install librosa
!pip install jiwer
!pip install transformers[torch]
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 6.8 MB/s eta 0:00:00

In [2]:
import torch
from torch.nn import Transformer, Linear, Embedding
from transformers import Wav2Vec2Processor, Wav2Vec2Config
from datasets import load_dataset, load_metric
from transformers import Trainer, TrainingArguments
import numpy as np
import json
import torch.nn as nn
from transformers import Wav2Vec2Model, Wav2Vec2Config
from transformers import PreTrainedTokenizer
import json
from transformers import BatchFeature
from huggingface_hub import notebook_login
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2Tokenizer

notebook_login()

In [3]:
# dataset = load_dataset("HamdanXI/lj_speech_DifferentStructure_removedVocabs")
dataset = load_dataset("HamdanXI/lj_speech_DifferentStructure_removedVocabs_prepared")
tokenizer = Wav2Vec2Tokenizer.from_pretrained("HamdanXI/lj_speech_DifferentStructure_removedVocabs-tokenizer")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/4620 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1680 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:736: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

'''
OR FOLLOW GUIDE

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=22050, padding_value=0.0, do_normalize=True, return_attention_mask=False)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
'''

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


'\nOR FOLLOW GUIDE\n\nfeature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=22050, padding_value=0.0, do_normalize=True, return_attention_mask=False)\nprocessor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)\n'

In [ ]:
'''
def prepare_dataset(batch):
    # Extract raw audio data from the 'audio' dictionary
    raw_audio = batch["audio"]["array"]

    # Extract input values. For Wav2Vec2, this would simply normalize the audio data
    input_values = processor(raw_audio, return_tensors="pt").input_values[0]

    # Encode the transcriptions to label ids using the processor in the target context
    with processor.as_target_processor():
        labels = processor(batch["text"], return_tensors="pt").input_ids[0]

    # Return the processed data
    return {
        "input_values": input_values,
        "labels": labels
    }

# Apply the prepare_dataset function on both train and test datasets
train_dataset = dataset["train"].map(prepare_dataset)
test_dataset = dataset["test"].map(prepare_dataset)

from datasets import DatasetDict

# Remove unwanted columns from train and test datasets
train_dataset = train_dataset.remove_columns(['audio', 'file', 'text'])
test_dataset = test_dataset.remove_columns(['audio', 'file', 'text'])

# Combine into a DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

from datasets import Dataset, Features, Sequence, Value

# 1. Extract data from the current dataset
input_values = test_dataset["input_values"]
labels_as_int = test_dataset["labels"]

# 2. Convert the labels to float64
labels_as_float = [list(map(float, label_list)) for label_list in labels_as_int]

# 3. Create a new dataset with the desired feature types
new_features = Features({
    'input_values': Sequence(Value(dtype='float32')),
    'labels': Sequence(Value(dtype='float64'))
})

new_dataset = Dataset.from_dict({
    'input_values': input_values,
    'labels': labels_as_float
}, features=new_features)

print(new_dataset.features)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': new_dataset
})

dataset_dict.push_to_hub("lj_speech_DifferentStructure_removedVocabs_prepared")
'''

In [26]:
class Wav2Vec2ForSeq2Seq(torch.nn.Module):
    def __init__(self, config: Wav2Vec2Config, vocab_size: int):
        super(Wav2Vec2ForSeq2Seq, self).__init__()
        self.wav2vec2 = Wav2Vec2Model(config)
        self.decoder = Transformer(
            d_model=config.hidden_size,
            nhead=config.num_attention_heads,
            num_encoder_layers=0,
            num_decoder_layers=config.num_hidden_layers
        )
        self.embedding = Embedding(vocab_size, config.hidden_size)
        self.output_layer = Linear(config.hidden_size, vocab_size)

    def forward(self, input_values, labels=None):
        encoder_outputs = self.wav2vec2(input_values).last_hidden_state
        if labels is not None:
            decoder_input = labels[:, :-1]
            embedded = self.embedding(decoder_input)
            decoder_outputs = self.decoder(embedded, encoder_outputs)
            logits = self.output_layer(decoder_outputs)
            return logits
        else:
            pass

class DataCollatorForSeq2Seq:
    def __init__(self, processor: Wav2Vec2Processor, model: Wav2Vec2ForSeq2Seq, padding=True):
        self.processor = processor
        self.model = model
        self.padding = padding

    def __call__(self, batch):
        input_values = self.processor.feature_extractor.pad(
            [feature["input_values"] for feature in batch],
            padding=self.padding,
            return_tensors="pt"
        )["input_values"]

        labels_with_tokens = []
        for feature in batch:
            labels_with_tokens.append([vocab_dict["[START]"]] + feature["labels"] + [vocab_dict["[END]"]])

        labels = self.processor.tokenizer.pad(
            labels_with_tokens,
            padding=self.padding,
            return_tensors="pt"
        )["input_ids"]

        return {
            "input_values": input_values,
            "labels": labels
        }

In [34]:
config = Wav2Vec2Config.from_pretrained("facebook/wav2vec2-base")
vocab_size = len(tokenizer)
model = Wav2Vec2ForSeq2Seq(config, vocab_size)

data_collator = DataCollatorForSeq2Seq(processor=processor, model=model)

wer_metric = load_metric("wer")

In [35]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred.label_ids[pred.label_ids == processor.tokenizer.pad_token_id] = -100
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

training_args = TrainingArguments(
    output_dir="lj_speech_DifferentStructure_removedVocabs-model",
    group_by_length=True,
    per_device_train_batch_size=32,
    evaluation_strategy="steps",
    num_train_epochs=2,
    fp16=True,
    save_steps=500,
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-4,
    weight_decay=0.005,
    warmup_steps=1000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
)

ValueError: ignored

In [ ]:
trainer.train()

trainer.push_to_hub()